In [8]:
import tensorflow as tf
import numpy as np

In [52]:
# 创建常量2
const = tf.constant(2.0, name='const')

# 创建变量
# b = tf.Variable(2.0, name='b')
b = tf.placeholder(tf.float32, [None, 1], name='b')
c = tf.Variable(1.0, dtype=tf.float32, name='c')

In [53]:
# 运算 operation
d = tf.add(b, c, name='d')
e = tf.add(c, const, name='e')
a = tf.multiply(d, e, name='a')

In [41]:
# tf 的所有变量都要初始化才能使用
# 1：定义初始化operation
init_op = tf.global_variables_initializer()
# 2：运行初始化operation
# session
with tf.Session() as sess:
    sess.run(init_op)
    #计算
    a_out = sess.run(a)
    print("变量 a = {}".format(a_out))

变量 a = 9.0


In [50]:
# 优化变量，使b 可以接收任意值
# 使用占位符（placeholder）来实现
# 第二个参数值为[None, 1]，其中None表示不确定，即不确定第一个维度的大小
# 第一维可以是任意大小。特别对应tensor数量(或者样本数量)，输入的tensor数目可以是32、64
b = tf.placeholder(tf.float32, [None, 1], name='b')

In [57]:
# 重新计算
# print(np.arange(0,10)[:,np.newaxis])
init_op_1 = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op_1)
    a_out = sess.run(a,feed_dict={b: np.arange(0,10)[:,np.newaxis]})
    print("a_out = {}".format(a_out))

a_out = [[ 3.]
 [ 6.]
 [ 9.]
 [12.]
 [15.]
 [18.]
 [21.]
 [24.]
 [27.]
 [30.]]


In [60]:
# 1. 加载数据：
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [59]:
# 2. 定义超参数和placeholder
learning_rate = 0.5
epochs = 10
batch_size = 100

# placeholder
# 输入图片为28 x 28 像素 = 784
x = tf.placeholder(tf.float32, [None, 784])
# 输出为0-9的one-hot编码
y = tf.placeholder(tf.float32, [None, 10])

In [61]:
# 3. 定义参数w和b
# hidden layer => w, b
W1 = tf.Variable(tf.random_normal([784, 300], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([300]), name='b1')
# output layer => w, b
W2 = tf.Variable(tf.random_normal([300, 10], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([10]), name='b2')

In [62]:
# 4. 构造隐层网络
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

In [63]:
# 5. 构造输出（预测值）
# 计算输出
y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))

J=−1m∑i=1m∑j=1nyijlog(y(i)j)+(1−y(i)jlog(1−y(i)j)

In [64]:
# 6. BP部分—定义loss 
# 对n个标签计算交叉熵
# 对m个样本取平均
y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped) + (1 - y) * tf.log(1 - y_clipped), axis=1))

In [66]:
# 7. BP部分—定义优化算法
# 创建优化器，确定优化目标
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

In [67]:
# 8. 定义初始化operation和准确率node
# init operator
init_op = tf.global_variables_initializer()

# 创建准确率节点
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [70]:
# 9 开始训练
# 创建session
with tf.Session() as sess:
    # 变量初始化
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels) / batch_size)
    print(total_batch)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            _, c = sess.run([optimizer, cross_entropy], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost = ", "{:.3f}".format(avg_cost))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))

550
Epoch: 1 cost =  0.553
Epoch: 2 cost =  0.205
Epoch: 3 cost =  0.148
Epoch: 4 cost =  0.113
Epoch: 5 cost =  0.088
Epoch: 6 cost =  0.071
Epoch: 7 cost =  0.056
Epoch: 8 cost =  0.043
Epoch: 9 cost =  0.034
Epoch: 10 cost =  0.026
0.9804
